Parameters selection

# 1. Velocity conversion: from Thymio units to rad/s

First goal is to convert the velocity of rotation of the wheels units from rad/s to the Thymio ones. Controller in fact will have as an input $v_r$ and $v_l$ in rad/s but will output them in Thymio Units to the plant.

<table><tr> 
<td> <img src="photo_2023-11-23_20-33-54.jpg" alt="photo_2023-11-23_20-33-54" style="width: 250px;"/> </td> 
 
</tr></table>

The code provided is the one used to estimate the conversion factor. In order to reduce the human error it was run fr many times. Fr every wheel we run this code (changing the target velocities for the two motors). The code counts the time untill one presses the central button. So, one person counts a certain amount of rounds and sign the number counted in the variable num_rotations. Then with some simple counts given the number of rotations and the time needed to do it, the velocity of rotation is calculated. The final coefficient is the average of the coefficients extracted.
In order to actually count the number of rounds we drew a green mark on the wheel.

In [1]:
from tdmclient import ClientAsync
import time
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

Node d4e6a400-466a-4476-a871-0ff69583670e

In [15]:
pressed = 0
await node.wait_for_variables({"button.center"})
Thymio_speed = 100
start = time.time()

while not pressed:
    v = {"motor.left.target": [0],
        "motor.right.target": [Thymio_speed],
            }
    await node.set_variables(v)
    pressed = node.v.button.center
    
end = time.time() 

v = {"motor.left.target": [0],
        "motor.right.target": [0],
            }   
await node.set_variables(v)

pressed = 0
print(end-start)

43.129746437072754


In [37]:
C_conv_toThymio_right= 67.60908181
C_conv_toThymio_left= 67.82946137


68.55734467759507


![Alt text](image.png)

In [16]:
await node.unlock()

# 2. Velocity conversion: from $v_r$ and $v_l$ to $v$ and $\omega$
https://www.cs.columbia.edu/~allen/F17/NOTES/icckinematics.pdf 

Linear velocity is the average of the speeds of the two wheels.
$ v = R\cdot \frac{v_{\text{r}} + v_{\text{l}}}{2}  $

Angular velocity depends on the difference in speed of the two wheels and the distance between them (wheelbase). If L is the distance between the centers of the two wheels, then:
$\omega = R \cdot\frac{v_{\text{r}} - v_{\text{l}}}{L} $ 


Given that we obtain:

$v_{r} = \frac{2v+\omega L}{2R}$ ,

$v_{l} = \frac{2v-\omega L}{2R}$


From the datasheet of the Thymio we have R and L parameters:
R=20 mm
L= 105 mm

In [ ]:
def convert_velocity(motor_right_speed, motor_left_speed,C_conv_toThymio_right, C_conv_toThymio_left, L, R):
    vr_rads= motor_right_speed/C_conv_toThymio_right
    vl_rads= motor_left_speed/C_conv_toThymio_left
    v=R* (vr_rads+vl_rads)/2   
    w=R*(vr_rads-vl_rads)/L
    return v, w


# 3. Esteem of Q: covariance matrix for the process noise

--> see Q_matrix_esteem.ipynb

$$
Q = 
\begin{bmatrix}
0.1366108 & 0 & 0 \\
0 & 0.13661 & 0 \\
0 & 0 & 0.00005
\end{bmatrix}
$$

In [1]:
import numpy as np

Q = np.array([[0.13661, 0, 0],[0 ,0.13661, 0], [0, 0, 0.00005]])

# 4. Esteem of R: covariance matrix for the measurement noise

Experiment set-up:

We printed four gridded A3 papers where the various positions where measured thanks to a reference system.
We moved the robot in various positions (x,y $\theta$) and we noted these coordinates while saving also the coordinates given by the camera:

In [1]:
import numpy as np
measured_coordinates=np.array([[0,0,0],[0,0,0],[0,0,0]]) #insert here the coordinates measured with the camera
real_coordinates=np.array([[0,0,0],[0,0,0],[0,0,0]]) #insert here the real coordinates of the Thymio

Moreover, we need the vector describing the translation between the two reference systems: the one of the camera and the one used for measuring the real positions:

In [ ]:
transformation_systems=np.array( 0, 0, 0)
#eventually you could need to tranfrom the data from the camera into the same units of the real coordinates

measured_coordinates= measured_coordinates-transformation_systems
error=measured_coordinates-real_coordinates

Then we can estimate the various elements we need:
var_x = np.var(error)